In [13]:
import os
import glob
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from niwidgets import NiftiWidget
import pandas as pd
import shutil
from skimage import morphology

In [52]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [7]:
#for f in glob.glob('C:/Users/e.lavrova/Desktop/AD_raw/sub-UCBM*/anat/Results/*.nii'):
#    if os.path.basename(f)[:3] != 'sub':
#        os.remove(f)

In [9]:
#for f in glob.glob('C:/Users/e.lavrova/Desktop/AD_raw/sub-UCBM*/anat/Results/*.mat'):
#    os.remove(f)

In [46]:
def get_stats(a):
    return np.median(a), np.percentile(a, 5), np.percentile(a, 95), np.min(a), np.max(a)

In [30]:
def normalize_img(a, a_min, a_max):
    a_in = (a>=a_min)*(a<=a_max)
    a_out_up = a>a_max
    a_out_down = a<a_min
    a_w = a*a_in+a_max*a_out_up+a_min*a_out_down
    return (a_max-a_w)/(a_max-a_min)

In [67]:
subs = ['AD001', 'AD002', 'AD003', 'AD004', 'AD005', 'AD006', 'AD007', 'AD008', 'AD009', 'AD010', 'AD011', 
        'AD012', 'AD013', 'AD014', 'AD015', 'AD016', 'AD017', 'AD018', 'AD019', 'AD020', 'AD021', 'AD022', 
        'AD023', 'AD024', 'HC001', 'HC002', 'HC003', 'HC004', 'HC005', 'HC006', 'HC007', 'HC008', 'HC009', 
        'HC010', 'HC011', 'HC012', 'HC013', 'HC014', 'HC015', 'HC016', 'HC017', 'HC018', 'HC019']

In [2]:
#s = []

#for sub in subs:

#    MT_path = 'C:/Users/e.lavrova/Desktop/AD_raw/sub-UCBMPM' + sub + '/anat/Results/sub-UCBMPM' + sub + '_echo-01_part-mag_PDw_MTsat.nii'
#    PD_path = 'C:/Users/e.lavrova/Desktop/AD_raw/sub-UCBMPM' + sub + '/anat/Results/sub-UCBMPM' + sub + '_echo-01_part-mag_PDw_PD.nii'
#    R1_path = 'C:/Users/e.lavrova/Desktop/AD_raw/sub-UCBMPM' + sub + '/anat/Results/sub-UCBMPM' + sub + '_echo-01_part-mag_PDw_R1.nii'
#    R2_path = 'C:/Users/e.lavrova/Desktop/AD_raw/sub-UCBMPM' + sub + '/anat/Results/sub-UCBMPM' + sub + '_echo-01_part-mag_PDw_R2s_OLS.nii'

#    MT_nii = nib.load(MT_path)
#    PD_nii = nib.load(PD_path)
#    R1_nii = nib.load(R1_path)
#    R2_nii = nib.load(R2_path)

#    MT = MT_nii.get_fdata()
    PD = PD_nii.get_fdata()
    R1 = R1_nii.get_fdata()
    R2 = R2_nii.get_fdata()

    MT_median, MT_p_5, MT_p_95, MT_min, MT_max = get_stats(MT)
    PD_median, PD_p_5, PD_p_95, PD_min, PD_max = get_stats(PD)
    R1_median, R1_p_5, R1_p_95, R1_min, R1_max = get_stats(R1)
    R2_median, R2_p_5, R2_p_95, R2_min, R2_max = get_stats(R2)
    
    s.append({'sub': sub, 
              'MT_median': MT_median, 'MT_p_5': MT_p_5, 'MT_p_95': MT_p_95, 'MT_min': MT_min, 'MT_max': MT_max, 
              'PD_median': PD_median, 'PD_p_5': PD_p_5, 'PD_p_95': PD_p_95, 'PD_min': PD_min, 'PD_max': PD_max, 
              'R1_median': R1_median, 'R1_p_5': R1_p_5, 'R1_p_95': R1_p_95, 'R1_min': R1_min, 'R1_max': R1_max,
              'R2_median': R2_median, 'R2_p_5': R2_p_5, 'R2_p_95': R2_p_95, 'R2_min': R2_min, 'R2_max': R2_max})

    MT_norm = normalize_img(MT, MT_p_5, MT_p_95)
    PD_norm = normalize_img(PD, PD_p_5, PD_p_95)
    R1_norm = normalize_img(R1, R1_p_5, R1_p_95)
    R2_norm = normalize_img(R2, R2_p_5, R2_p_95)

    print (MT.shape, PD.shape, R1.shape, R2.shape)

    dim = MT.shape
    th_0 = int(round(dim[0]/2))
    th_1 = int(round(dim[1]/2))
    th_2 = int(round(dim[2]/2))

    comb = np.zeros(dim)

    comb[:th_0, :th_1, :th_2] = MT_norm[:th_0, :th_1, :th_2].copy()
    comb[th_0:, :th_1, :th_2] = PD_norm[th_0:, :th_1, :th_2].copy()
    comb[:th_0, th_1:, :th_2] = R1_norm[:th_0, th_1:, :th_2].copy()
    comb[th_0:, th_1:, :th_2] = R2_norm[th_0:, th_1:, :th_2].copy()

    comb[:th_0, :th_1, th_2:] = R2_norm[:th_0, :th_1, th_2:].copy()
    comb[th_0:, :th_1, th_2:] = R1_norm[th_0:, :th_1, th_2:].copy()
    comb[:th_0, th_1:, th_2:] = PD_norm[:th_0, th_1:, th_2:].copy()
    comb[th_0:, th_1:, th_2:] = MT_norm[th_0:, th_1:, th_2:].copy()

    comb_nii = nib.Nifti1Image(comb, affine=MT_nii.affine)
    nib.save(comb_nii, sub+'.nii') 

"s = []\n\nfor sub in subs:\n\n    MT_path = 'C:/Users/e.lavrova/Desktop/AD_raw/sub-UCBMPM' + sub + '/anat/Results/sub-UCBMPM' + sub + '_echo-01_part-mag_PDw_MTsat.nii'\n    PD_path = 'C:/Users/e.lavrova/Desktop/AD_raw/sub-UCBMPM' + sub + '/anat/Results/sub-UCBMPM' + sub + '_echo-01_part-mag_PDw_PD.nii'\n    R1_path = 'C:/Users/e.lavrova/Desktop/AD_raw/sub-UCBMPM' + sub + '/anat/Results/sub-UCBMPM' + sub + '_echo-01_part-mag_PDw_R1.nii'\n    R2_path = 'C:/Users/e.lavrova/Desktop/AD_raw/sub-UCBMPM' + sub + '/anat/Results/sub-UCBMPM' + sub + '_echo-01_part-mag_PDw_R2s_OLS.nii'\n\n    MT_nii = nib.load(MT_path)\n    PD_nii = nib.load(PD_path)\n    R1_nii = nib.load(R1_path)\n    R2_nii = nib.load(R2_path)\n\n    MT = MT_nii.get_fdata()\n    PD = PD_nii.get_fdata()\n    R1 = R1_nii.get_fdata()\n    R2 = R2_nii.get_fdata()\n\n    MT_median, MT_p_5, MT_p_95, MT_min, MT_max = get_stats(MT)\n    PD_median, PD_p_5, PD_p_95, PD_min, PD_max = get_stats(PD)\n    R1_median, R1_p_5, R1_p_95, R1_min,

In [50]:
s = pd.DataFrame(s)

In [53]:
s

,sub,MT_median,MT_p_5,MT_p_95,MT_min,MT_max,PD_median,PD_p_5,PD_p_95,PD_min,PD_max,R1_median,R1_p_5,R1_p_95,R1_min,R1_max,R2_median,R2_p_5,R2_p_95,R2_min,R2_max
0,AD001,0.083939,-0.545591,1.530219,-5.0,5.0,4.549447,0.00000,92.238427,-4123.566895,199.999954,0.481843,0.0,1.678455,0.0,2.0,4.211295,-15.241208,80.084896,-99.288208,227.485550
1,AD002,0.041120,-0.347978,1.450559,-5.0,5.0,5.594870,0.00000,86.264597,-10449.629883,199.998810,0.348137,0.0,1.276268,0.0,2.0,2.152465,-12.866025,86.460801,-127.969658,228.520645
2,AD003,0.098868,-0.274565,1.488528,-5.0,5.0,5.021571,0.00000,96.205469,-11312.113281,199.999588,0.305175,0.0,1.155988,0.0,2.0,3.445365,-10.585813,86.090081,-128.465012,240.043243
3,AD004,0.069371,-0.288590,1.491533,-5.0,5.0,3.864589,0.00000,90.809899,-9934.640625,199.998398,0.428845,0.0,1.237357,0.0,2.0,3.618298,-8.803478,88.878988,-131.428513,234.794098
4,AD005,0.064286,-0.540548,1.620162,-5.0,5.0,5.342301,0.00000,98.763100,-1656.836182,199.999435,0.435250,0.0,1.710023,0.0,2.0,4.892246,-12.973015,95.515570,-111.156479,229.183365
5,AD006,0.024695,-0.847740,2.433056,-5.0,5.0,5.988741,0.00000,104.417245,-5029.573730,199.998993,0.382040,0.0,1.618070,0.0,2.0,7.212496,-10.543698,95.506289,-131.733948,221.872711
6,AD007,0.050017,-0.345846,1.493418,-5.0,5.0,4.471119,0.00000,94.982644,-182430.953125,199.998581,0.424704,0.0,1.672353,0.0,2.0,3.512859,-9.284327,95.708550,-128.360947,226.474396
7,AD008,0.082900,-0.284609,1.652684,-5.0,5.0,4.421748,-298.95450,94.018493,-19493.355469,199.999863,0.367129,0.0,1.425885,0.0,2.0,3.817459,-10.213117,96.807170,-124.952591,231.297546
8,AD009,0.055634,-0.340821,1.472402,-5.0,5.0,4.937571,0.00000,93.213854,-60493.707031,199.999603,0.381313,0.0,1.690830,0.0,2.0,3.080857,-12.991399,89.478161,-118.920647,235.613617
9,AD010,0.068729,-0.591336,3.842659,-5.0,5.0,5.520108,-90.13847,99.905779,-72467.617188,199.998352,0.419503,0.0,1.598542,0.0,2.0,5.207611,-8.971544,90.503375,-145.022720,223.442383


MT: AD023  
PD: AD008, AD010   
R2: AD020, HC016

In [63]:
#for folder in os.listdir("C:/Users/e.lavrova/Desktop/AD_raw")[1:]:
#    os.mkdir("C:/Users/e.lavrova/Desktop/AD_raw/derivatives/hmri/"+folder+"/anat")
#    os.mkdir("C:/Users/e.lavrova/Desktop/AD_raw/derivatives/hmri/"+folder+"/fmap")

In [3]:
#anat_files = glob.glob("C:/Users/e.lavrova/Desktop/AD_raw/sub-*/anat/Results/sub*")
#print (len(anat_files))
#anat_files

In [66]:
#for f in anat_files:
#    sub = os.path.basename(f)[:15]
#    shutil.move(f, 'C:/Users/e.lavrova/Desktop/AD_raw/derivatives/hmri/'+sub+'/anat/'+os.path.basename(f))

In [68]:
#for sub in subs:
#    f = 'C:/Users/e.lavrova/Desktop/AD_raw/sub-UCBMPM'+sub+'/anat/Results/Supplementary'
#    shutil.move(f, 'C:/Users/e.lavrova/Desktop/AD_raw/derivatives/hmri/sub-UCBMPM'+sub+'/anat/Supplementary')

In [159]:
def save_masked(path, mask):
    head, tail = os.path.split(path)
    img_nii = nib.load(path)
    img = img_nii.get_fdata()
    m_img = img*mask
    m_img_nii = nib.Nifti1Image(m_img, affine=img_nii.affine)
    nib.save(m_img_nii, head + '/mask'+tail)

In [ ]:
subname = "AD001"

In [194]:
c1_path = "C:/Users/e.lavrova/Desktop/AD/sub-UCBMPM" + subname + "/anat/Supplementary/c1sub-UCBMPM" + subname + "_echo-01_part-mag_PDw_T1w_OLSfit_TEzero.nii"
c2_path = "C:/Users/e.lavrova/Desktop/AD/sub-UCBMPM" + subname + "/anat/Supplementary/c2sub-UCBMPM" + subname + "_echo-01_part-mag_PDw_T1w_OLSfit_TEzero.nii"
c3_path = "C:/Users/e.lavrova/Desktop/AD/sub-UCBMPM" + subname + "/anat/Supplementary/c3sub-UCBMPM" + subname + "_echo-01_part-mag_PDw_T1w_OLSfit_TEzero.nii"
c5_path = "C:/Users/e.lavrova/Desktop/AD/sub-UCBMPM" + subname + "/anat/Supplementary/c5sub-UCBMPM" + subname + "_echo-01_part-mag_PDw_T1w_OLSfit_TEzero.nii"

c1_nii = nib.load(c1_path)
c2_nii = nib.load(c2_path)
c3_nii = nib.load(c3_path)
c5_nii = nib.load(c5_path)

c1 = c1_nii.get_fdata()
c2 = c2_nii.get_fdata()
c3 = c3_nii.get_fdata()
c5 = c5_nii.get_fdata()

icv = ((c1 + c2 + c3)>0)
mask = ((c1 + c2 + c3 + (c5>0.9))>0)

plt.subplot(1, 4, 1)
plt.imshow(icv[:, 128, :])

plt.subplot(1, 4, 2)
plt.imshow(morphology.binary_dilation(icv)[:, 128, :])

plt.subplot(1, 4, 3)
plt.imshow(mask[:, 128, :])

plt.subplot(1, 4, 4)
plt.imshow(morphology.closing(mask, selem=np.ones((11, 11, 11)))[:, 128, :])

plt.show()

#icv = morphology.binary_dilation(icv, selem=np.ones((9, 9, 9)))
#icv_nii = nib.Nifti1Image(icv.astype(int), affine=c1_nii.affine)
#nib.save(icv_nii, "C:/Users/e.lavrova/Desktop/AD/sub-UCBMPM" + subname + "/anat/Supplementary/icvsub-UCBMPM" + subname + "_echo-01_part-mag_PDw_T1w_OLSfit_TEzero.nii") 

mask = morphology.closing(mask, selem=np.ones((11, 11, 11)))
mask = morphology.remove_small_holes(mask, area_threshold=10000, connectivity=3)
mask_nii = nib.Nifti1Image(mask.astype(int), affine=c1_nii.affine)
nib.save(mask_nii, "C:/Users/e.lavrova/Desktop/AD/sub-UCBMPM" + subname + "/anat/Supplementary/masksub-UCBMPM" + subname + "_echo-01_part-mag_PDw_T1w_OLSfit_TEzero.nii")

save_masked("C:/Users/e.lavrova/Desktop/AD/sub-UCBMPM" + subname + "/anat/sub-UCBMPM" + subname + "_echo-01_part-mag_PDw_MTsat.nii", mask)
save_masked("C:/Users/e.lavrova/Desktop/AD/sub-UCBMPM" + subname + "/anat/sub-UCBMPM" + subname + "_echo-01_part-mag_PDw_PD.nii", mask)
save_masked("C:/Users/e.lavrova/Desktop/AD/sub-UCBMPM" + subname + "/anat/sub-UCBMPM" + subname + "_echo-01_part-mag_PDw_R1.nii", mask)
save_masked("C:/Users/e.lavrova/Desktop/AD/sub-UCBMPM" + subname + "/anat/sub-UCBMPM" + subname + "_echo-01_part-mag_PDw_R2s_OLS.nii", mask)

In [197]:
c7_path = "C:/Users/e.lavrova/Desktop/AD/sub-UCBMPMHC002/anat/c7masksub-UCBMPMHC002_echo-01_part-mag_PDw_MTsat.nii"
c7_nii = nib.load(c7_path)
c7 = c7_nii.get_fdata()

c7_corr = ((c7>0.1) != (c3>0.5))*c7

c7_corr_nii = nib.Nifti1Image(c7_corr.astype(float), affine=c7_nii.affine)
nib.save(c7_corr_nii, "C:/Users/e.lavrova/Desktop/AD/sub-UCBMPMHC002/anat/tc7masksub-UCBMPMHC002_echo-01_part-mag_PDw_MTsat.nii")